In [1]:
# Sharks and Fish

In this tutorial we will look at building a model from scratch using pre-defined MathML equations, generating code from the model, and solving the model.

We are going to create a simple model representing the population dynamics of two species - one a predator (sharks), and the other their prey (fish).
The population of fish can only grow when there are not too many sharks eating them, the rate at which the fish population grows depends on how many fish are available for breeding.
At the same time, the population of sharks will depend on how much food is available in the fish population.
In maths this relationship can be written:

$$
c = a + 2.0
$$
$$
\frac{dy_s}{dt} =f(sharks, fishes, time) = a y_s + b y_s y_f
$$
$$
\frac{dy_f}{dt} =f(sharks, fishes, time) = c y_f + d y_s y_f
$$

where the constants are defined as $(a, b, d)=(-0.8, 0.3, -0.6)$ and we'll use the initial conditions of $y_s(t=0)=1.0$ and
$y_f(t=0)=2.0$. $c$ is what we call, in a libCellML context, a computed constant.

In order to model these unusual populations you'll need to create your own custom units, enter these governing equations in MathML syntax, and to use the **Generator** functionality to create files that are able to be solved using a numerical integrator in C++ or Python.

To start we import libCellML.

In [2]:
import libcellml

Then we create a model with a name, and a single component also with a name which we add to the model.

In [3]:
model = libcellml.Model("sharks_v_fish")
sea_component = libcellml.Component("sea")
model.addComponent(sea_component)

True

A point of note is that CellML encapsulates math in MathML 2.0 only.
For various reasons even though MathML 3.0 and MathML 4.0 exist, CellML only accepts MathML 2.0 math.
Further to this point, CellML 2.0 only allows a reduced set of the MathML 2.0 standard.
But don't worry, the allowed MathML elements is sufficient to describe your favourite ODE or DAE model.

Looking at the top equation first, the MathML2 representation of $c = a + 2.0$ is:

```xml
<apply>
  <eq/>
  <ci>c</ci>
  <apply>
    <plus/>
    <ci>a</ci>
    <cn cellml:units="per_month">2.0</cn>
  </apply>
</apply>
```

We won't go in to the MathML too deeply (or at all) but hopefully you can see just how this equation has been encoded in MathML
We can see here that the *apply* element is a building block in MathML.
This element applies an operator to any number of operands.
The operators themselves are generally self-evident as to their purpose.

The eagle eyed amoung you will of course notice the *cellml:units="per_month"* attribute on the constant element.
We must define the units for this constant so that our equations maintain their units consistency.
We will bring the units full circle soon.

Here is the second equation:

```xml
<apply>
  <eq/>
  <apply>
    <diff/>
    <bvar>
      <ci>time</ci>
    </bvar>
    <ci>y_s</ci>
  </apply>
  <apply>
    <plus/>
    <apply>
       <times/>
       <ci>a</ci>
       <ci>y_s</ci>
    </apply>
    <apply>
      <times/>
      <ci>b</ci>
      <ci>y_s</ci>
      <ci>y_f</ci>
    </apply>
  </apply>
</apply>
```

And the third:

```xml
<apply>
  <eq/>
  <apply>
    <diff/>
    <bvar>
      <ci>time</ci>
    </bvar>
    <ci>y_f</ci>
  </apply>
  <apply>
    <plus/>
    <apply>
      <times/>
      <ci>c</ci>
      <ci>y_f</ci>
    </apply>
    <apply>
      <times/>
      <ci>d</ci>
      <ci>y_s</ci>
      <ci>y_f</ci>
    </apply>
  </apply>
</apply>
```

The MathML added to a component must be a valid MathML block of text (according to the MathML 2 DTD).
The math in a component can be a series of valid MathML blocks of text or just a single MathML document.
To this end we use the MathML header as:
```xml
<math xmlns="http://www.w3.org/1998/Math/MathML" xmlns:cellml="http://www.cellml.org/cellml/2.0#">
```
You will of course notice the addition of the CellML namespace definition, we need to define this namespace so that we can set the appropriate units for any constants that appear in the math.
We close the MathML document with:
```xml
</math>
```

To add the math to our component we shall load the MathML document from the *resources* directory.

In [4]:
with open("resources/sharks_and_fish.mathml") as fh:
    mathml = fh.read()

sea_component.setMath(mathml)

A big part of CellML is using variables to define an interface to the model, we need to define these variables in the component with the appropriate units.
The variables that are used in the math all need to be represented by variables in the component.
This means we need to define CellML variables for the following variables in the math:
* time
* y_s
* y_f
* a
* b
* c
* d

In [5]:
# Define and add variables to the component.
time = libcellml.Variable("time")
sea_component.addVariable(time)
sharks = libcellml.Variable("y_s")
sea_component.addVariable(sharks)
fish = libcellml.Variable("y_f")
sea_component.addVariable(fish)
a = libcellml.Variable("a")
sea_component.addVariable(a)
b = libcellml.Variable("b")
sea_component.addVariable(b)
c = libcellml.Variable("c")
sea_component.addVariable(c)
d = libcellml.Variable("d")
sea_component.addVariable(d)

True

Now we need to define the units for our variables in the model. 
Having units consistency in a model is a very important for spotting inconsistencies in the model, libCellML has powerful units checking capabilities to help spot these issues.
You can of course define every variable as having the units *dimensionless* to bypass this check but this practice is heavily discouraged.

LibCellML defines a number of units already, the S.I. units plus a collection of other common units, see the documentation [here](https://libcellml.org/documentation/v0.6.3/api/classlibcellml_1_1Units#classlibcellml_1_1Units_1a0d2387a42a6df513cfa55d6ee8e6bf09) for a full list of the pre-defined units.
For this model we need to define the following additional units:
* month
* per_month
* number_of_sharks
* thousands_of_fish
* per_shark_month
* per_1000fish_month

In [6]:
# Define the required units, add them to the model, and set them to the appropriate variable.
month = libcellml.Units("month")
month.addUnit("second", 0, 1, 2592000)
model.addUnits(month)
time.setUnits(month)

per_month = libcellml.Units("per_month")
per_month.addUnit("month", -1)
model.addUnits(per_month)
a.setUnits(per_month)
c.setUnits(per_month)

number_of_sharks = libcellml.Units("number_of_sharks")
model.addUnits(number_of_sharks)
sharks.setUnits(number_of_sharks)

thousands_of_fish = libcellml.Units("thousands_of_fish")
model.addUnits(thousands_of_fish)
fish.setUnits(thousands_of_fish)

per_shark_month = libcellml.Units("per_shark_month")
per_shark_month.addUnit("per_month")
per_shark_month.addUnit("number_of_sharks", -1)
model.addUnits(per_shark_month)
b.setUnits(per_shark_month)


per_1000fish_month = libcellml.Units("per_1000fish_month")
per_1000fish_month.addUnit("per_month")
per_1000fish_month.addUnit("thousands_of_fish", -1)
model.addUnits(per_1000fish_month)
d.setUnits(per_1000fish_month)

Now we need to check that our model is valid before we try and generate any code.
The first thing the generator does is check the validity of the model, so if it isn't valid we aren't going to get any further.

In [7]:
validator = libcellml.Validator()
validator.validateModel(model)
if validator.errorCount() == 0:
    print("Our model is valid.")
else:
    print("Houston we have a problem!")


Our model is valid.


Whilst the model is valid it will not be able to be used for generating code as we have not set the initial conditions for the model leaving it under-defined.
To define the initial conditions we set the initial values for the variables that require it.
The constant variables $a$, $b$, and $d$ can be set to $−0.8$, $0.3$, and $−0.6$ respectively.
The variables $y_s$ and $y_f$ can be initialised to $1.0$ and $2.0$.

In [8]:
# Set initial values for all required variables.
a.setInitialValue(-0.8)
b.setInitialValue(0.3)
d.setInitialValue(-0.6)
sharks.setInitialValue(1.0)
fish.setInitialValue(2.0)

Now our model is ready for analysing, and we can use the analysed model to generate the code we can use to simulate the model.

In [9]:
analyser = libcellml.Analyser()
analyser.analyseModel(model)

In [10]:
analyser_model = analyser.model()
profile = libcellml.GeneratorProfile(libcellml.GeneratorProfile.Profile.PYTHON)
generator = libcellml.Generator()
generator.setProfile(profile)
generator.setModel(analyser_model)

In [11]:
code = generator.implementationCode()
print(code)

# The content of this file was generated using the Python profile of libCellML 0.6.3.

from enum import Enum
from math import *


__version__ = "0.4.0"
LIBCELLML_VERSION = "0.6.3"

STATE_COUNT = 2
VARIABLE_COUNT = 4


class VariableType(Enum):
    VARIABLE_OF_INTEGRATION = 0
    STATE = 1
    CONSTANT = 2
    COMPUTED_CONSTANT = 3
    ALGEBRAIC = 4


VOI_INFO = {"name": "time", "units": "month", "component": "sea", "type": VariableType.VARIABLE_OF_INTEGRATION}

STATE_INFO = [
    {"name": "y_f", "units": "thousands_of_fish", "component": "sea", "type": VariableType.STATE},
    {"name": "y_s", "units": "number_of_sharks", "component": "sea", "type": VariableType.STATE}
]

VARIABLE_INFO = [
    {"name": "a", "units": "per_month", "component": "sea", "type": VariableType.CONSTANT},
    {"name": "c", "units": "per_month", "component": "sea", "type": VariableType.COMPUTED_CONSTANT},
    {"name": "b", "units": "per_shark_month", "component": "sea", "type": VariableType.CONSTANT},
    {"name"